In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


In [8]:
data = pd.read_excel('../input/sarcasm-detection/Dataset.xlsx', sheet_name='Full Data')
# Keeping only the neccessary columns
print(data.columns)
data = data[['Comments','0/1']]
data.head()

Index(['Comments', 'Sarcasm_Y_N', '0/1', 'Platform'], dtype='object')


,Comments,0/1
0,Sorry No Comments ..I came here to read commen...,1
1,Hi guys. I'm so happy and proud of myself and ...,1
2,Hahahaha your intelligence ðŸ˜œðŸ˜œðŸ˜œ,1
3,Aqsa Naveed we were proud backbenchers ðŸ˜œðŸ˜...,1
4,Hemant extraordinary sketcherðŸ˜..right Ujjaw...,1


In [10]:
# data = data[data.sentiment != "Neutral"]
data['Comments'] = data['Comments'].apply(lambda x: x.lower())
data['Comments'] = data['Comments'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['0/1'] == 1].size)
print(data[ data['0/1'] == 0].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Comments'].values)
X = tokenizer.texts_to_sequences(data['Comments'].values)
X = pad_sequences(X)

18520
26140


In [11]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 688, 128)          256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 688, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
Y = pd.get_dummies(data['0/1']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 688) (14961, 2)
(7369, 688) (7369, 2)


In [17]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
 - 611s - loss: 0.3360 - acc: 0.8654
Epoch 2/5
 - 588s - loss: 0.3109 - acc: 0.8707
Epoch 3/5
 - 584s - loss: 0.2854 - acc: 0.8796
Epoch 4/5
 - 587s - loss: 0.2707 - acc: 0.8866
Epoch 5/5
 - 596s - loss: 0.2899 - acc: 0.8776


In [18]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.34
acc: 0.87


In [20]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 68.0577849117175 %
neg_acc 97.49144811858609 %


In [ ]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")